stolen from https://github.com/GSNCodes/ArUCo-Markers-Pose-Estimation-Generation-Python/blob/main/pose_estimation.py

and https://stackoverflow.com/questions/55816902/finding-the-intersection-of-two-circles

4corners0
width
0.1571640692466518
0.15769984424676786
length
0.1822147497672536
0.1818431585400156

4corners1
width 
0.15173707172872597
0.15800170256710422
length
0.18222924825472253
0.1620446584729262

In [1]:
import cv2
import numpy as np
import pathlib
import os
import cv2.aruco as aruco
import argparse
import sys
import imutils
import math

In [2]:
def save_coefficients(mtx, dist, path):
    '''Save the camera matrix and the distortion coefficients to given path/file.'''
    cv_file = cv2.FileStorage(path, cv2.FILE_STORAGE_WRITE)
    cv_file.write('K', mtx)
    cv_file.write('D', dist)
    # note you *release* you don't close() a FileStorage object
    cv_file.release()

def load_coefficients(path):
    '''Loads camera matrix and distortion coefficients.'''
    # FILE_STORAGE_READ
    cv_file = cv2.FileStorage(path, cv2.FILE_STORAGE_READ)

    # note we also have to specify the type to retrieve other wise we only get a
    # FileNode object back instead of a matrix
    camera_matrix = cv_file.getNode('K').mat()
    dist_matrix = cv_file.getNode('D').mat()

    cv_file.release()
    return [camera_matrix, dist_matrix]
def find_center(markerCorner):
    (topLeft, topRight, bottomRight, bottomLeft) = markerCorner[0]

    # convert each of the (x, y)-coordinate pairs to integers
    topRight = (int(topRight[0]), int(topRight[1]))
    bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
    bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
    topLeft = (int(topLeft[0]), int(topLeft[1]))

    # find center marker
    cX = int((topLeft[0] + bottomRight[0]) / 2.0)
    cY = int((topLeft[1] + bottomRight[1]) / 2.0)
    return cX, cY

def find_distance(a, b):
    return math.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2)

def get_intersections(x0, y0, r0, x1, y1, r1):
    # circle 1: (x0, y0), radius r0
    # circle 2: (x1, y1), radius r1

    d=math.sqrt((x1-x0)**2 + (y1-y0)**2)
    
    # non intersecting
    if d > r0 + r1 :
        return None
    # One circle within other
    if d < abs(r0-r1):
        return None
    # coincident circles
    if d == 0 and r0 == r1:
        return None
    else:
        a=(r0**2-r1**2+d**2)/(2*d)
        h=math.sqrt(r0**2-a**2)
        x2=x0+a*(x1-x0)/d   
        y2=y0+a*(y1-y0)/d   
        x3=x2+h*(y1-y0)/d     
        y3=y2-h*(x1-x0)/d 

        x4=x2-h*(y1-y0)/d
        y4=y2+h*(x1-x0)/d
        
        return [x3, y3], [x4, y4]

In [3]:
mtx, dist = load_coefficients('calibration_aruco.yml')

In [34]:
def pose_estimation(frame, aruco_dict_type, matrix_coefficients, distortion_coefficients):

    '''
    frame - Frame from the video stream
    matrix_coefficients - Intrinsic matrix of the calibrated camera
    distortion_coefficients - Distortion coefficients associated with your camera
    return:-
    frame - The frame with the axis drawn on it
    '''

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.aruco_dict = cv2.aruco.Dictionary_get(aruco_dict_type)
    parameters = cv2.aruco.DetectorParameters_create()


    corners, ids, rejected_img_points = cv2.aruco.detectMarkers(gray, cv2.aruco_dict,parameters=parameters,
        cameraMatrix=matrix_coefficients,
        distCoeff=distortion_coefficients)
    
    rvecs = []
    tvecs = []
        # If markers are detected
    if len(corners) > 0:
        print(str(ids))
        for i in range(0, len(ids)):
            
            
            # Estimate pose of each marker and return the values rvec and tvec---(different from those of camera coefficients)
            rvec, tvec, markerPoints = cv2.aruco.estimatePoseSingleMarkers(corners[i], 0.02, matrix_coefficients,
                                                                       distortion_coefficients)
            # Draw a square around the markers
            cv2.aruco.drawDetectedMarkers(frame, corners) 

            # Draw Axis
            cv2.aruco.drawAxis(frame, matrix_coefficients, distortion_coefficients, rvec, tvec, 0.01)
            
            # draw marker number
            (topLeft, topRight, bottomRight, bottomLeft) = corners[i][0]
            topLeft = (int(topLeft[0]), int(topLeft[1]))
            cv2.putText(image, str(ids[i]),
            (topLeft[0] - 15, topLeft[1]), cv2.FONT_HERSHEY_SIMPLEX,
            0.5, (0, 255, 0), 2)
            
            entry = [ids[i][0], rvec[0][0]]
            rvecs.append(entry)
            entry = [ids[i][0], tvec[0][0]]
            tvecs.append(entry)
    length = -1
    width = -1
    if len(ids) == 5:
        centers = {}
        for i in range(len(ids)):
            point = corners[i]
            currID = ids[i]
            x, y = find_center(point)
            print('id ' + str(currID) + ' centered at ' + str(x) + ' ' + str(y))
            centers[currID[0]] = (x, y)
        
        length = find_distance([centers[3][0], centers[3][1]], [centers[2][0], centers[2][1]])
        width  = find_distance([centers[0][0], centers[0][1]], [centers[1][0], centers[1][1]])
        
    
    return frame, rvecs, tvecs, zip(ids, corners), length, width

In [35]:
image = cv2.imread("img1.png")
image = imutils.resize(image, width=600)
arucoDict = cv2.aruco.DICT_4X4_50

output, rvecs, tvecs, corners, length, width = pose_estimation(image, arucoDict, mtx, dist)
cv2.imshow('rotate', output)
cv2.waitKey(0)

[[4]
 [0]]


102

In [36]:
cv2.destroyAllWindows()

In [37]:
rvecs

[[4, array([ 1.50501965,  1.45055049, -0.98228721])],
 [0, array([ 1.48138185, -1.48317256,  1.02059836])]]

In [23]:
tvecs
tvec_list = [0]*5
for index, tvec in tvecs:
    tvec_list[index] = tvec
tvec_list

[array([-0.09680448, -0.06370044,  0.89519308]),
 array([ 0.05316002, -0.06649869,  0.91814865]),
 array([-0.09327158, -0.0076726 ,  0.72182671]),
 array([ 0.05837023, -0.01071023,  0.7660994 ]),
 array([-0.0512938 , -0.02901526,  0.80957514])]

In [24]:
#WIDTH using euclidian distance
a = np.linalg.norm(tvecs[3][1] - tvecs[2][1])
print(a)
b = np.linalg.norm(tvecs[0][1] - tvecs[1][1])
print(b)
width = (a+b)/2
width

0.10297920280067402
0.15800170256710422


0.13049045268388912

In [25]:
#LENGTH using euclidian distance
a = np.linalg.norm(tvecs[0][1] - tvecs[2][1])
print(a)
b = np.linalg.norm(tvecs[1][1] - tvecs[3][1])
print(b)
length = (a+b)/2
length

0.09958629460845107
0.20869195327157838


0.15413912394001472

In [26]:
#for every corner, find the intersection
corners =[(0, 0), (width, 0), (0, length), (width, length)]
possible_points = []
point_tvec = tvec_list[4]
for i in range(4):
    for j in range(i+1, 4):
        x0, y0 = corners[i]
        r0 = np.linalg.norm(point_tvec-tvec_list[i])
        x1, y1 = corners[j]
        r1 = np.linalg.norm(point_tvec-tvec_list[j])
        returned = get_intersections(x0, y0, r0, x1, y1, r1)
        if returned != None:
            point1, point2 = returned
            possible_points.append(point1)
            possible_points.append(point2)

In [27]:
possible_points

[[0.013520761028131274, -0.10208773300785219],
 [0.013520761028131274, 0.10208773300785219],
 [0.06569920381242363, 0.07929899638630976],
 [-0.06569920381242363, 0.07929899638630976],
 [0.09480025007596544, 0.04021975627719243],
 [0.024023165474172006, 0.10013792353580583],
 [0.09956628844215418, 0.1521430578852541],
 [-0.014462957989956997, 0.05560865697663355],
 [0.2410186846505153, 0.10902900501727629],
 [0.019962220717262946, 0.10902900501727629],
 [0.04863871326795856, 0.06723857362254824],
 [0.04863871326795856, 0.24103967425748118]]

In [13]:
block_width = width/2
block_length = length/2

possible_locations = []
for x, y in possible_points:
    if x < 0 or y < 0:
        continue 
    x = math.floor(x/block_width)
    y = math.floor(y/block_length)
    if x > 1 or y > 1:
        continue
    possible_locations.append([x, y])

In [14]:
possible_locations

[[1, 0], [1, 0], [1, 0], [1, 0]]

In [31]:
"""
Cleans possible points computed with tvecs and returns estimated spot in row major order
tldr; vote on most likely location after intersection of 2 circles
INPUTS
    width - 23, 10 average
    length - 02, 13 average
    box_size - an integer, box robot lives in will be size x size, should be square...
    possible_points - possible points computed from intersection mess

"""

def clean_points_and_vote(width, length, box_size, possible_points):
    block_width = width/box_size
    block_legnth = length/box_size
    possible_coords = [0]*box_size*box_size
    max_votes = 0
    max_vote_index = -1;
    for x, y in possible_points:
        if x < 0 or y < 0:
            continue 
        x = math.floor(x/block_width)
        y = math.floor(y/block_length)
        if x > box_size-1 or y > box_size-1:
            continue
        print(str(x) + ' ' + str(y))
        possible_coords[ y * box_size + x] += 1
        if possible_coords[ y * box_size + x] > max_votes:
            max_votes = possible_coords[ y * box_size + x]
            max_vote_index =  y * box_size + x
    return max_vote_index

In [32]:
"""
for a 3x3 matrix in row major order

0 1 2
3 4 5
6 7 8

"""
prediction = clean_points_and_vote(width, length, 3, possible_points)

0 1
1 0
2 0
0 1
2 1
0 1
1 0
1 2


In [30]:
prediction

3